# Instalação do software e introdução à biblioteca pySMT

Nesta UC vamos estudar metodologias e ferramentas para modelar e verificar propriedades lógicas de sistemas.
As ferramentas que vamos usar são [SMT solvers](https://en.wikipedia.org/wiki/Satisfiability_modulo_theories#SMT_solvers) e a ferramenta de programação inteira [SCIP](https://www.scipopt.org).

A linguagem de programação que vamos usar é o [Python](https://www.python.org) e as aulas práticas serão desenvolvidas dentro de um [Jupyter](https://jupyter.org) notebook, a ser executado na plataforma
[Anaconda](https://www.anaconda.com).
Usaremos a biblioteca [OR-Tools](https://developers.google.com/optimization) para fazer a interface para o SCIP e a biblioteca [pySMT](https://github.com/pysmt/pysmt) para fazer a interface com os SMT solvers. Também poderá vir a ser útil usar a bibloteca de grafos [NetworkX](https://networkx.org).

# Instalação do software


Os passos que a seguir se apresentam são para instalar o Anaconda, criar um "environment" (logica) especifico para esta UC onde instalamos o Python 3.10, o Jupyter, a biblioteca pySMT e os SMT solvers Z3 e MathSAT, e ainda as bibliotecas OR-Tools e NetworkX.

Estes passos correspondem à instalação em MacOS.

1. Instalar o Anaconda a partir do site  https://www.anaconda.com


2. Iniciar o Conda package manager.

        ~/opt/anaconda3/bin/conda init
        
3. Criar um ambiente específico (chamado "logica").

        conda create -n logica python=3.10  
        
4. Ativar o ambiente "logica".

        conda activate logica
        
5. Instalar o Jupyter nesse ambiente.
  
        conda install jupyter 

6. Instalar a biblioteca pySMT e os SMT solvers (Z3 e MathSAT) nesse ambiente.

        pip install pysmt
        pysmt-install --z3
        pysmt-install --msat
        
    **Nota:** Quem estiver em Windows, ao tentar instalar o MathSAT, provavelmente vai ter o seguinte erro: `Microsoft Visual C++ 14.0 or greater is required`. Para o resolver têm de ir ao endereço (https://visualstudio.microsoft.com/downloads/?q=build+tools#build-tools-for-visual-studio-2022), fazer download e, aquando da instalação, selecionar a opção do `C++ Buildtools`.
    
        
7. Instalar as bibliotecas OR-Tools e NetworkX nesse ambiente.

        pip install ortools
        pip install networkx
        

Para arrancar com o Jupyter, na linha de comando, fazer `jupyter notebook`

# Breve introdução à utilização de SMT solvers com  a biblioteca pySMT

O problema SMT (*Satisfiability Modulo Theories*) é o problema de satisfatibilidade para lógica de primeira ordem
no âmbito de alguma teoria lógica específica - uma teoria lógica que fixa as interpretações de certos predicados e símbolos de função. Dito de outra forma, restringe-se a satisfatibilidade a uma classe específica de modelos, numa lógica de primeira ordem tipificada e com igualdade.
Os *SMT solvers* são ferramentas que visam responder ao problema SMT. Como o problema não é decidível, pode ser necessário (ou conveniente) restringir a classe de fórmulas em consideração a um fragmento (isto é, restrição sintática) adequado.

Os SMT solvers são o motor central de muitas ferramentas de análise e verificação de programas, geração de casos de teste, bounded model checking of SW, planeamento, etc. 
Existem muitos SMT solvers disponíveis. Por exemplo: Z3, MathSAT, CVC4, Yices, entre outros. Alguns são direcionados a teorias específicas; 
muitos suportam o formato SMT-LIB (um formato textural normalizado de input/output para SMT solvers); 
muitos fornecem recursos não padronizados. Mais informação [aqui](https://en.wikipedia.org/wiki/Satisfiability_modulo_theories#SMT_solvers).

## A biblioteca pySMT

A biblioteca [pySMT](https://github.com/pysmt/pysmt) permite que um programa em Python comunique com vários SMT solvers, tendo por base uma linguagem comum. Permite assim codificar um problema de forma independente do SMT solver, e correr o mesmo problema com vários SMT solvers.
A documentação do pySMT pode ser encontrada em https://pysmt.readthedocs.io/en/latest/index.html.

Vamos explorar alguns exemplos disponibilizados no manual do pySMT e propor novos desafios.

## Primeiros exemplos


O pySMT é altamente estruturado, mas oferece uma API simplificada que disponibiliza as funcionalidades para a utilização usual de um SMT solver. Essa API agrupa num único módulo todas as funções para construir fórmulas, verificar a satisfatibilidade e recuperar instâncias do solver. Esse módulo é o `pysmt.shortcuts`.

Neste primeiro exemplo vamos testar a satisfatibilidade de duas fórmulas proposicionais: $(A \wedge \neg B)$ e $(A \wedge \neg A)$.

Para isso, primeiro precisamos criar duas novas variáveis $A$ e $B$. As variáveis PySMT são chamadas de “símbolos” e são criadas usando a função `Symbol()` que recebe como entrada um nome de variável e, opcionalmente, um tipo. Por omissão, os símbolos são Booleanos.

Para este exemplo, vamos precisar das seguintes funções: `Symbol`, `And`, `Not`, `is_sat` e `get_model`.

In [ ]:
from pysmt.shortcuts import Symbol, And, Not, is_sat, get_model

varA = Symbol("A")    # Default type is Boolean
varB = Symbol("B")
f = And(varA, Not(varB))

res = is_sat(f)
print("f := %s is SAT? %s" % (f, res))

O teste de satisfatibilidade da fórmula pode ser feito com a função `is_sat()`. É possivel explicitar o SMT solver que queremos usar.

In [ ]:
resZ3 = is_sat(f,solver_name="z3")
resMSAT = is_sat(f,solver_name="msat")

print("f := %s is SAT (z3)? %s" % (f, resZ3))

print("f := %s is SAT (msat)? %s" % (f, resMSAT))

Como a fórmula é satisfazível, isso significa que existe uma interpretação para seus símbolos não lógicos que torna a fórmula verdadeira. Ou seja, que existe um modelo para a fórmula.

Para sabermos qual o modelo que o solver encontrou podemos usar a função `get_model()`. Se a fórmula é satifazível, esta função devolve um modelo para a fórmula (isto é, uma espécie de dicionário que mapeia cada variável lógica no seu valor), caso contrário, devolve `None`.

In [ ]:
print("Model:")
model = get_model(f)
print(model)

Vamos agora gerar a fórmula $A \wedge \neg A$ ilustrando como podemos fazer uma substituição com o método `substitute()`. Neste caso vamos substituir a variável $B$ por $A$ na fórmula `f`.

In [ ]:
g = f.substitute({varB:varA})

res = is_sat(g)
print("g := %s is SAT? %s" % (g, res))

print(get_model(g))


Vamos agora trabalhar com a teoria dos inteiros, para saber se é possível arranjar valores inteiros $x$ e $y$ entre 1 e 10, tal que $x+y > 10$ e $x-y\leq 5$. 

Para criar variáveis inteiras temos que indicar o seu tipo. Os tipos estão definidos no módulo `pysmt.typing` de onde temos que importar o tipo `INT`.

In [ ]:
from pysmt.shortcuts import Symbol, is_sat, get_model, And
from pysmt.typing import INT

x = Symbol("x", INT)
y = Symbol("y", INT)

formula = And(1<=x , x<=10 , 1<=y , y<=10 , x+y>10 , x-y<=5)

print(get_model(formula))

Ao importar `pysmt.shortcuts` a notação infixa fica disponível. No entanto, podemos usar os operadores textuais importando-os de `pysmt.shortcuts`. Isto por vezes torna o código mais claro, distingindo entre os operadores do Python e do SMT.

In [ ]:
from pysmt.shortcuts import Symbol, is_sat, get_model, And, LE, GE, GT, Int, Not, Or, Equals
from pysmt.typing import INT

x = Symbol("x", INT)
y = Symbol("y", INT)

formula = And(LE(Int(1),x) , GE(Int(10),x) , LE(Int(1),y) , GT(x+y,Int(10)), LE(x-y,Int(5)))

print(get_model(formula))

### Exercício 1

Será que esta é a única solução para este problema? Como poderiamos tirar partido do solver para saber isso?

In [ ]:
# completar


Em vez de definir uma variável de cada vez, podemos usar as listas por compreensão do Python para definir vários símbolos. 
As compreensões são tão comuns no pySMT que operadores n-ários (como `And()`, `Or()`, `Plus()`) podem aceitar um objecto iterável (por exemplo, listas ou gerador). Vejamos o seguinte exemplo.

## Hello World

O problema é o seguinte: 
queremos associar a cada  uma das letra que compõem as palavras HELLO e WORLD, um valor inteiro entre 1 e 10, de forma a que `H+E+L+L+O = W+O+R+L+D = 25`. Será que isso é possivel?

Vejamos a seguinte formalização do problema.

In [ ]:
from pysmt.shortcuts import Symbol, LE, GE, Int, And, Equals, Plus, Solver, is_sat, get_model
from pysmt.typing import INT

hello = [Symbol(s, INT) for s in "hello"]
world = [Symbol(s, INT) for s in "world"]

letters = set(hello+world)
print(letters)

domains = And(And(LE(Int(1), l),
                  GE(Int(10), l)) for l in letters)

sum_hello = Plus(hello)
sum_world = Plus(world)

problem = And(Equals(sum_hello, sum_world),
              Equals(sum_hello, Int(25)))

formula = And(domains, problem)

print("Serialization of the formula:")
print(formula)

Se a fórmula for muito grande, certas subfórmulas podem ser mostradas como `...`. Se quiser garantir que vê sempre a fórmula toda use o método `serialize()`.

In [ ]:
print(formula.serialize())

### Exercício 2

Veja se o problema tem solução e, se tiver, apresente uma solução.

In [ ]:
# completar


Estas funções de atalho são muito úteis em situações pontuais. Contudo, a forma mais usual de utilização de um SMT solver consiste em criar uma instância de um `Solver` e trabalhar com ele de forma incremental. Isto faz-se através da função `Solver()`, e 
pode ser usado dentro de um contexto (com a instrução `with`) para lidar automaticamente com a destruição do solver e dos recursos associados. 
É possível especificar qual o solver que queremos executar e/ou a teoria lógica em que queremos trabalhar.

Depois de criar o solver, podemos adicionar restrições de forma incremental (com o método `add_assertion()`), testar a satisfatibilidade desse conjunto de restrições (com o método `solve()`), inspecionar o modelo, etc.
No exemplo abaixo lançamos o solver Z3 e, em primeiro lugar, verificamos se a fórmula `domain` é satisfazível. Depois, em caso afirmativo, continuamos a resolver o problema. Repare que, neste exemplo, acedemos ao valor de cada símbolo com o método `get_value()`. Porém, também podemos obter o modelo usando o método `get_model()`.

In [ ]:
with Solver(name="z3") as solver:
    solver.add_assertion(domains)
    if not solver.solve():
        print("Domain is not SAT!!!")
        exit()
    solver.add_assertion(problem)
    if solver.solve():
        for l in letters:
            print("%s = %s" %(l, solver.get_value(l)))
    else:
        print("No solution found")

### Exercício 3

Altere o código acima de forma a usar o método `get_model()`, apresentado o resultado com o mesmo formato.

In [ ]:
# completar
 

### Exercício 4

Com o pySMT é possível executar o mesmo código usando diferentes SMT solvers. No nosso exemplo, podemos especificar qual o solver que queremos executar alterando a maneira como o instanciamos. Use o MathSAT para resolver o problema anterior. Veja se a solução apresentada é igual à do Z3.

In [ ]:
# completar
 

Podemos lançar um solver simplemente com a indicação da teoria lógica com que queremos trabalhar.
Neste caso, o solver é escolhido entre os solvers instalados que suportam essa lógica. Se não existir é gerada uma 
exceção (`NoSolverAvailableErro`).
É claro que também podemos indicar o solver.

No exemplo a seguir escolhe-se a lógica `QF_LIA` (*Quantifier-Free Linear Integer Arithmetic*).

In [ ]:
from pysmt.shortcuts import Symbol, LE, GE, Int, And, Equals, Plus, Solver
from pysmt.typing import INT

hello = [Symbol(s, INT) for s in "hello"]
world = [Symbol(s, INT) for s in "world"]

letters = set(hello+world)

domains = And([And(LE(Int(1), l),
                   GE(Int(10), l)) for l in letters])

sum_hello = Plus(hello)
sum_world = Plus(world)

problem = And(Equals(sum_hello, sum_world),
              Equals(sum_hello, Int(36)))

formula = And(domains, problem)

with Solver(logic="QF_LIA") as solver:
    solver.add_assertion(domains)
    if not solver.solve():
        print("Domain is not SAT!!!")
        exit()
    solver.add_assertion(problem)
    if solver.solve():
        for l in letters:
            print("%s = %s" %(l, solver.get_value(l)))
    else:
        print("No solution found")

### Exercício 5

O Cryptarithms é um jogo que consiste numa equação matemática entre números desconhecidos, cujos dígitos são representados por letras. Cada letra deve representar um dígito diferente e o dígito inicial de um número com vários dígitos não deve ser zero.

Queremos saber os dígitos a que correspondem as letras envolvidas na seguinte equação:
```
TWO + TWO = FOUR
```
Podemos modelar o problema numa teoria de inteiros. Cada letra dá origem a uma variável inteira ($T$,$W$,$O$,$F$,$U$, e $R$) e para representar a equação acima representamos cada parcela por uma expressão aritmética onde cada letra é multiplicada pelo seu “peso específico” (em base 10).

Resolver este problema equivale a resolver o seguinte sistema de equações:
$$
\left\{
\begin{array}{l}
0 \le T \le 9\\
\cdots\\
0 \le R \le 9\\
T \neq W \neq O \neq F \neq U \neq R \\
T \neq 0\\
F \neq 0\\
(100 \times T + 10 \times W + O) + (100 \times T + 10 \times W + O) = 1000 \times F + 100 \times O + 10 \times U + R
\end{array}
\right.
$$

Use o Z3 para resolver este problema. Nota: poderá ser útil usar o operador `AllDifferent`.

In [ ]:
# completar


### Exercício 6

Considere o seguinte enigma:
```
- If the unicorn is mythical, then it is immortal.
- If the unicorn is not mythical, then it is a mortal mammal.
- If the unicorn is either immortal or a mammal, then it is horned. 
- The unicorn is magical if it is horned.

Given these constraints:
   - Is the unicorn magical? 
   - Is it horned? 
   - Is it mythical?
```
Modele o problema em lógica proposicional criando uma variável proposicional para cada caracteristica dos unicornios.
Use um SMT solver para o resolver.

**Sugestão:** Resolva o problema com o auxílio de 5 variáveis proposicionais, correspondentes às 5 propriedades 
    dos unicórnios. Relembre que a afirmação $A_1, \ldots, A_n \models B$ é válida se e só se o conjunto de 
    restrições $\{A_1, \ldots, A_n, \neg B\}$ é inconsistente. Tire proveito dos métodos `push()` e `pop()` para 
    responder às várias questões usando de forma incremental o mesmo solver.

In [ ]:
# completar
